<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Install-gentle" data-toc-modified-id="Install-gentle-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Install gentle</a></span></li><li><span><a href="#Compute-audio_duration,-bad_transcript,-missing_transcript" data-toc-modified-id="Compute-audio_duration,-bad_transcript,-missing_transcript-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Compute audio_duration, bad_transcript, missing_transcript</a></span></li><li><span><a href="#Test-aligning-transcript-with-local-video" data-toc-modified-id="Test-aligning-transcript-with-local-video-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Test aligning transcript with local video</a></span></li><li><span><a href="#Test-aligning-transcript-using-scanner-pipeline" data-toc-modified-id="Test-aligning-transcript-using-scanner-pipeline-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Test aligning transcript using scanner pipeline</a></span></li><li><span><a href="#Complete-transcript-loading" data-toc-modified-id="Complete-transcript-loading-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Complete transcript loading</a></span></li><li><span><a href="#Analyze-result" data-toc-modified-id="Analyze-result-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Analyze result</a></span></li><li><span><a href="#Check-audio-time->-frame-time" data-toc-modified-id="Check-audio-time->-frame-time-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Check audio time &gt; frame time</a></span></li></ul></div>

# Install gentle 
gentle is a 3rd party library for audio-transcript aligment, we need gentle installed in the docker image 

1.
git clone https://github.com/scanner-research/gentle

2.
bash ./install.sh (takes ~30min)   

# Compute audio_duration, bad_transcript, missing_transcript
We need to pre-compute audio duration instead of video duration(number of frames) which will be used in the AudioSource kernel; We also want to pre-filter raw transcripts to make sure they are complete (longer than certain amount of the whole video) 

In [ ]:
from esper.util import *
import pickle
from query.models import Video
from tqdm import tqdm
import multiprocessing

# videos = Video.objects.filter(path__contains='2018')

video_ids = open('/app/data/video_list_2019.txt', 'r').read().split('\n')
videos = Video.objects.filter(id__in=video_ids)
len(videos)

In [ ]:
num_threads = 80
pool = multiprocessing.Pool(num_threads)
num_video_t = len(videos) // num_threads
video_list_t = []
for i in range(num_threads):
    if i != num_threads - 1:
        video_list_t.append(videos[i*num_video_t: (i+1)*num_video_t])
    else:
        video_list_t.append(videos[i*num_video_t:])

pkl_path_list = pool.map(get_video_field_t, video_list_t)

In [ ]:
# pkl_path_list = open('/app/tmp/pkl_path_list.txt', 'r').read().split()

# result_final = pickle.load(open('/app/data/addtional_field_all.pkl', 'rb'))
result_final = {}
for pkl_path in pkl_path_list:
    result = pickle.load(open(pkl_path, 'rb'))
    for key, val in result.items():
        result_final[key] = val
print(len(result_final))
pickle.dump(result_final, open('/app/data/addtional_field_2019.pkl', 'wb'))

In [ ]:
result_final = pickle.load(open('/app/data/addtional_field_all.pkl', 'rb'))
cnt = 0
for key, res in result_final.items():
    if not res['valid_transcript']:
        cnt += 1
print(cnt, cnt / len(result_final))

# Test aligning transcript with local video
Test the transcript aligment class locally without using scanner at all

In [ ]:
import pickle
from query.models import Video
import scannerpy
import os

get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')

In [ ]:
result = scannerpy.Database().summarize()

In [ ]:
# set test video list
video_list = ['FOXNEWSW_20160604_140000_Cost_of_Freedom_Dash_to_the_Clash']

In [ ]:
# set srt extension
for video_name in video_list:
    video = Video.objects.filter(path__contains=video_name)[0]
    video.srt_extension = 'word'
    video.save()

In [ ]:
from scannertools.transcript_alignment import TranscriptAligner

res_stats = {}
for video_name in video_list:
    print(video_name)
    # download video
    video_path = os.path.join('../data/videos/', video_name+'.mp4')
    if not os.path.exists(video_path):
        gs_path = os.path.join('gs://esper/tvnews/videos/', video_name+'.mp4')
        cmd = 'gsutil cp ' + gs_path + ' ' + '../data/videos/'
        os.system(cmd)
    print('Downloading video done')
    
    # run alignment
    aligner = TranscriptAligner(win_size=300, seg_length=60, max_misalign=10, num_thread=64, estimate=True,
                            media_path=video_path,
                            transcript_path=os.path.join('/app/data/subs10/', video_name),
                            align_dir='/app/data/subs/orig/')
    res = aligner.run_all()
#     res_stats[video_name] = res
#     pickle.dump(res_stats, open('/app/result/test_align_100_hard.pkl', 'wb'))

# Test aligning transcript using scanner pipeline
Test the transcript aligment using scanner pipeline. Please also refer to /app/esper/transcript_aligment.py for more instructions. We use that script to run large jobs

In [ ]:
from scannertools import audio
from scannertools.transcript_alignment import align_transcript_pipeline
from tqdm import tqdm
SEG_LENGTH = 60

In [ ]:
# set test video list
video_list = ['FOXNEWSW_20160604_140000_Cost_of_Freedom_Dash_to_the_Clash']
videos = [Video.objects.filter(path__contains=video_name)[0] for video_name in video_list]

# videos = Video.objects.filter(threeyears_dataset=True).all()
addtional_field = pickle.load(open('/app/data/addtional_field_2019.pkl', 'rb'))
# videos = [video for video in videos if addtional_field[video.id]['valid_transcript']]
# videos = videos[:30000]

In [ ]:
# check database
db = scannerpy.Database()
videos_committed = []
for video in tqdm(videos):
    table_name = '{}_align_transcript'.format(video.path)
    table = db.table(table_name)
    if not table.committed():
        print(video.item_name())
    else:
        videos_committed.append(video)
videos = videos_committed
len(videos)

In [ ]:
# load audios from videos
audios = [audio.AudioSource(video.for_scannertools(), 
                                frame_size=SEG_LENGTH, 
                                duration=addtional_field[video.id]['audio_duration']) 
              for video in videos]

# set up transcripts 
captions = [audio.CaptionSource('tvnews/subs2019/'+video.item_name(), 
                                max_time=addtional_field[video.id]['audio_duration'] , 
                                window_size=SEG_LENGTH) 
            for video in videos]

In [ ]:
# set up run opts
run_opts = {'pipeline_instances_per_node': 96, 'checkpoint_frequency': 5}

# set up align opts
align_opts = {'win_size': 300,
                  'seg_length' : 60,
                  'max_misalign' : 10,
                  'num_thread' : 1,
                  'estimate' : False,
                  'align_dir' : None,
                  'res_path' : None,
}

In [ ]:
result = align_transcript_pipeline(db=db, audio=audios, captions=captions, cache=False, 
                                                   run_opts=run_opts, align_opts=align_opts)

# Complete transcript loading
Before this step, run /app/esper/transcript_aligment.py with BATCH_LOAD to dump results from scanner database. Then use this part to collect staus of the aligment result

In [ ]:
import pickle

In [ ]:
updates = open('/app/tmp/align_new_6.log', 'r').read().split('\n')
print(updates[:5])

updates = [line.split(' ') for line in updates[5:-1]]
len(updates)

In [ ]:
res_stats = pickle.load(open('/app/result/align_stats_final_2019.pkl', 'rb'))
# res_stats = {}

In [ ]:
for path, r in updates:
    video = Video.objects.filter(path=path)[0]
    res_stats[video.id] = {'word_missing': float(r)}
len(res_stats)

In [ ]:
pickle.dump(res_stats, open('/app/result/align_stats_final_2019.pkl', 'wb'))

In [ ]:
video_ids = open('/app/data/video_list_2019.txt', 'r').read().split('\n')
videos = Video.objects.filter(id__in=video_ids)
addtional_field = pickle.load(open('/app/data/addtional_field_2019.pkl', 'rb'))

In [ ]:
for video in videos:
    if video.id in res_stats and res_stats[video.id]['word_missing'] <= 0.2:
        addtional_field[video.id]['aligned_transcript'] = True
    else:
        addtional_field[video.id]['aligned_transcript'] = False

In [ ]:
pickle.dump(addtional_field, open('/app/data/addtional_field_2019.pkl', 'wb'))

In [ ]:
cnt = 0
for id in addtional_field:
    if addtional_field[id]['aligned_transcript']:
        cnt += 1
print(cnt)

In [ ]:
with open('/app/result/aligned2019.txt', 'w') as f:
    for video_id in sorted(addtional_field):
        video = Video.objects.filter(id=video_id)[0]
        if addtional_field[video_id]['aligned_transcript']:
            f.write(video.item_name() + '\n')
f.close()

# Analyze result

In [ ]:
align_stats_first = pickle.load(open('/app/result/align_stats_first.pkl', 'rb'))
align_stats_second = pickle.load(open('/app/result/align_stats_second.pkl', 'rb'))

In [ ]:
videos = Video.objects.all()
cnt = 0
video_list = []
clean_file = open('/app/result/clean_subs.txt', 'w')
for video in videos:
    if video.id in align_stats_second and align_stats_second[video.id]['word_missing'] <= 0.2:
#         if cnt < 100:
#             print(video.id, res_stats[video.id]['word_missing'])
#             video_list.append(video.id)
        cnt += 1
        clean_file.write(video.item_name() + '\n')
print(len(videos), cnt)
# print(video_list)
clean_file.close()

In [ ]:
# merge second into first
align_stats_final = {id: res for id, res in align_stats_first.items() if res['word_missing'] < 0.2}
for id, res in align_stats_second.items():
    align_stats_final[id] = res
res_list_first = [res['word_missing'] if res['word_missing'] > 0 else 0 for id, res in align_stats_first.items() ]
res_list_final = [res['word_missing'] if res['word_missing'] > 0 else 0 for id, res in align_stats_final.items() if res['word_missing'] < 1]
res_list_first.sort()
res_list_final.sort()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(res_list_first)
plt.plot(res_list_final)
plt.xlabel('num of videos')
plt.ylabel('mis-aligned ratio')

# Check audio time > frame time

In [ ]:
# set test video list
videos = Video.objects.all()
addtional_field = pickle.load(open('/app/data/addtional_field_all.pkl', 'rb'))
videos = [video for video in videos if addtional_field[video.id]['valid_transcript']]

In [ ]:
cnt = 0
for video in videos:
    audio_time = addtional_field[video.id]['audio_duration']
    frame_time = video.num_frames / video.fps
    if audio_time / frame_time > 1.1 or audio_time / frame_time < 0.9:
        cnt += 1
cnt